In [1]:
import numpy as np
import pandas as pd

# Lab 01
David Gao (I work this alone)
## 1  Data Import and Summary
##### 1.

In [48]:
covid = pd.read_csv(r'C:\UW\SPR 2022\INFO 370\time-series-covid19-deaths-us.csv', sep='\t')
covid.head(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/17/22,3/18/22,3/19/22,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,209,210,210,210,210,210,210,209,209,209
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,668,669,669,669,672,673,673,673,674,674
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,94,94,94,94,94,97,97,97,97,97
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,100,100,100,100,100,101,101,101,101,101
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,228,230,230,230,230,230,230,230,233,233


##### 2.

In [148]:
rows,columns = covid.shape
print('rows:', rows, '; columns:', columns)

rows: 3342 ; columns: 807


##### 3.

In [8]:
data_types = covid.dtypes
data_types

UID          int64
iso2        object
iso3        object
code3        int64
FIPS       float64
            ...   
3/22/22      int64
3/23/22      int64
3/24/22      int64
3/25/22      int64
3/26/22      int64
Length: 807, dtype: object

##### 4.

In [10]:
sub_covid = covid[['FIPS', 'Admin2', 'Province_State','3/1/20', '3/15/21', '3/26/22']]
sub_covid.sample(5)

,FIPS,Admin2,Province_State,3/1/20,3/15/21,3/26/22
1244,24011.0,Caroline,Maryland,0,21,80
153,5067.0,Jackson,Arkansas,0,36,60
577,15007.0,Kauai,Hawaii,0,1,29
1650,29213.0,Taney,Missouri,0,91,223
2353,42045.0,Delaware,Pennsylvania,0,1262,1856


## 2  Explore deaths in Washington
##### 1.

In [11]:
WA_deaths = sub_covid[sub_covid.Province_State == 'Washington']
WA_deaths.sample(5)

,FIPS,Admin2,Province_State,3/1/20,3/15/21,3/26/22
3163,53037.0,Kittitas,Washington,0,30,60
3175,53059.0,Skamania,Washington,0,1,12
3161,53033.0,King,Washington,1,1440,2655
3182,53071.0,Walla Walla,Washington,0,64,140
3156,53023.0,Garfield,Washington,0,4,7


##### 2.

In [12]:

WA_deaths['growth'] = WA_deaths['3/26/22'] - WA_deaths['3/15/21']
WA_deaths.sample(5)

C:\Users\David.Gao\AppData\Local\Temp\ipykernel_15444\2566550715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WA_deaths['growth'] = WA_deaths['3/26/22'] - WA_deaths['3/15/21']


,FIPS,Admin2,Province_State,3/1/20,3/15/21,3/26/22,growth
3168,53047.0,Okanogan,Washington,0,36,120,84
3151,53013.0,Columbia,Washington,0,4,15,11
3177,53063.0,Spokane,Washington,0,583,1352,769
3167,53045.0,Mason,Washington,0,25,137,112
3155,53021.0,Franklin,Washington,0,103,211,108


##### 3.
There is a warning as the operation of adding a row "growth" was made directly on a partial view the orignial data. To avoid this warning, we can make a copy wity `.copy()` before modifying data.

In [69]:
WA_deaths = sub_covid[sub_covid.Province_State == 'Washington'].copy()
WA_deaths['growth'] = WA_deaths['3/26/22'] - WA_deaths['3/15/21']
WA_deaths.shape

(41, 7)

##### 4.

In [51]:
growth = WA_deaths["growth"]
type(growth)

pandas.core.series.Series

##### 5.
The index is the line number of original U.S. covid data. And it is not a useful index. I think FIPS can be a better index as it is unique for every counties.

##### 6.

In [56]:
growth.index = WA_deaths['FIPS']
growth.head(5)

FIPS
53001.0     16
53003.0     37
53005.0    255
53007.0     98
53009.0    129
Name: growth, dtype: int64

## 3  Data interpretation
##### 1.

In [142]:
top = growth.sort_values()
index = top.index
index.shape

(41,)

In [145]:
counties = []
for i in index[31:]:
    temp = WA_deaths[WA_deaths.FIPS == i][['Admin2','growth']]
    counties.append(temp)

counties

[      Admin2  growth
 3162  Kitsap     245,
       Admin2  growth
 3147  Benton     255,
        Admin2  growth
 3152  Cowlitz     270,
         Admin2  growth
 3179  Thurston     322,
       Admin2  growth
 3185  Yakima     341,
      Admin2  growth
 3150  Clark     546,
          Admin2  growth
 3176  Snohomish     567,
        Admin2  growth
 3177  Spokane     769,
       Admin2  growth
 3172  Pierce     787,
      Admin2  growth
 3161   King    1215]

The counties with top 10 deaths growth are King, Pierce, Spokane, Snohomish, Clark, Yakima, Thurston, Cowlitz, Benton, and Kitsap.

##### 2.
The efficacy of social distancing measures in Seattle did not work vary well, as King county has largest number of deaths growth.